In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import calendar
import time

import uszipcode

plt.style.use('ggplot')
start_time = dt.datetime.now()
%matplotlib inline

In [2]:
%%time
data = pd.read_csv('card transactions.csv', dtype = object)
data = data.iloc[:, :-8].copy()
data.shape

CPU times: total: 219 ms
Wall time: 246 ms


(96753, 10)

In [3]:
data['Recnum'] = data['Recnum'].astype(int)
data['Date'] = pd.to_datetime(data['Date'])
data['Amount'] = data['Amount'].astype(float)
data['Fraud'] = data['Fraud'].astype(int)

data.dtypes

Recnum                        int32
Cardnum                      object
Date                 datetime64[ns]
Merchnum                     object
Merch description            object
Merch state                  object
Merch zip                    object
Transtype                    object
Amount                      float64
Fraud                         int32
dtype: object

# Data Preparation

## Fill in Missing Data (Imputation)

### Filtering

In [4]:
data_copy = data.copy().sort_values('Amount', ascending = False)[1:].query("Transtype == 'P'").copy()

# rename cols with space in it
data_copy.rename(columns={'Merch description': 'Merch_description', 
                          'Merch state': 'Merch_state',
                          'Merch zip': 'Merch_zip'}, inplace=True)

### Merch_state

#### Merch state with zip code

In [5]:
engine = uszipcode.SearchEngine()

for i in data_copy.query('Merch_state.isna() & Merch_zip.notna()').index:
    try:
        zipcode = engine.by_zipcode(int(data_copy.loc[i, 'Merch_zip']))
        data_copy.loc[i, 'Merch_state'] = zipcode.state
    except AttributeError:
        print('{} is not in the USA.'.format(data_copy.loc[i, 'Merch_zip']))
        data_copy.loc[i, 'Merch_state'] = 'Unknown'
        pass

00002 is not in the USA.
60528 is not in the USA.
60528 is not in the USA.
60528 is not in the USA.
60528 is not in the USA.
23080 is not in the USA.
23080 is not in the USA.
00006 is not in the USA.
00002 is not in the USA.
00002 is not in the USA.
86899 is not in the USA.
86899 is not in the USA.
00006 is not in the USA.
00008 is not in the USA.
00006 is not in the USA.
50823 is not in the USA.
65132 is not in the USA.
00009 is not in the USA.
00006 is not in the USA.
00006 is not in the USA.
00002 is not in the USA.
00009 is not in the USA.
00009 is not in the USA.
86899 is not in the USA.
48700 is not in the USA.
48700 is not in the USA.
00001 is not in the USA.
01400 is not in the USA.
00001 is not in the USA.
00009 is not in the USA.
00003 is not in the USA.


#### Merch state with <font style=color:Red> no zipcode </font>

In [6]:
# Using the Merch state of 'mode of Merch description' to replace all nan
mode = data_copy['Merch_description'].mode().tolist()[0]
state_mode = data_copy[data_copy['Merch_description'] == mode].iloc[0,5]
empty_after_fill_ms = data_copy[data_copy['Merch_state'].isna()]
for i in empty_after_fill_ms.index:
    data_copy.loc[i, 'Merch_state'] = state_mode

### Merchnum

In [7]:
# if Merchnum = '0', replace with NaN
data_copy['Merchnum'] = data_copy['Merchnum'].replace('0', np.nan)

# We have no clear clue for deciding the Merchant number, 
# so we will use the Merchnum of 'mode of Merch description' to replace all nan
mode = data_copy['Merch_description'].mode().tolist()[0]
num_mode = data_copy[data_copy['Merch_description'] == mode].iloc[0,3]
empty_after_fill_mn = data_copy[data_copy['Merchnum'].isna()]
for i in empty_after_fill_mn.index:
    data_copy.loc[i, 'Merchnum'] = num_mode

### Merch zip

In [8]:
# We will use the Merch zip of 'mode of Merch description' to replace all nan
mode = data_copy['Merch_description'].mode().tolist()[0]
zip_mode = data_copy[data_copy['Merch_description'] == mode].iloc[0,6]
empty_after_fill_mz = data_copy[data_copy['Merch_zip'].isna()]
for i in empty_after_fill_mz.index:
    data_copy.loc[i, 'Merch_zip'] = zip_mode

In [9]:
data = data_copy.copy()
del data_copy

# Creating Variables

In [10]:
# right alignment
data['Cardnum'] = data['Cardnum'].apply(lambda x:'{0:0>10}'.format(x))
data['Merch_zip'] = data['Merch_zip'].apply(lambda x:'{0:0>5}'.format(x))
data['Merchnum'] = data['Merchnum'].apply(lambda x:'{0:0>13}'.format(x))

## Day of week

In [11]:
# Day of Week Target Encoding
data['dow'] = data['Date'].apply(lambda x: calendar.day_name[x.weekday()])

In [16]:
# Separate the OOT
train_set = data.query("Date < '2010-11-01'")

#Statistical Smoothing
c=4; nmid=20; y_avg=train_set['Fraud'].mean()
y_dow=train_set.groupby('dow')['Fraud'].mean()
num=train_set.groupby('dow').size()
y_dow_smooth=y_avg+(y_dow-y_avg)/(1+np.exp(-(num-nmid)/c))
data['dow_risk']=data.dow.map(y_dow_smooth)

# delete files that we don't need any more to free up memory
del train_set

## Making new entities

In [13]:
# Make new entities for variables
data['Merch_num_des'] = data.Merchnum + data.Merch_description
data['Merch_num_state'] = data.Merchnum + data.Merch_state
data['Merch_num_zip'] = data.Merchnum + data.Merch_zip
data['Merch_des_state'] = data.Merch_description + data.Merch_state
data['Merch_des_zip'] = data.Merch_description + data.Merch_zip
data['Merch_state_zip'] = data.Merch_state + data.Merch_zip
data['Merch_num_des_state'] = data.Merchnum + data.Merch_description + data.Merch_state
data['Merch_num_des_zip'] = data.Merchnum + data.Merch_description + data.Merch_zip
data['Merch_num_state_zip'] = data.Merchnum + data.Merch_state + data.Merch_zip
data['Merch_des_state_zip'] = data.Merch_description + data.Merch_state + data.Merch_zip
data['Merch_all_info'] = data.Merchnum + data.Merch_description + data.Merch_state + data.Merch_zip

In [14]:
# No Merch_zip because zip can be duplicate between State anyway
entity_list = ['Merchnum', 'Merch_description', 'Merch_state', 'Merch_zip', 'Merch_num_des', 'Merch_num_state', 'Merch_num_zip',
             'Merch_des_state', 'Merch_des_zip', 'Merch_state_zip', 'Merch_num_des_state', 'Merch_num_des_zip',
             'Merch_num_state_zip', 'Merch_des_state_zip', 'Merch_all_info']

card_list = ['card_' + entity for entity in entity_list]

for i in entity_list:
    data['card_'+i] = data.Cardnum + data[i]

In [15]:
attributes = ['Cardnum'] + entity_list + card_list
attributes.remove('Merch_state')
attributes.remove('Merch_zip')
attributes

# I decided to remove Merch_state and Merch_zip. It requires a lot of computing power but it doesn't make much sense.
# e.g. Like, You count the number of trans in the same state regardless of card and merchant?
# That's basically state GDP

['Cardnum',
 'Merchnum',
 'Merch_description',
 'Merch_num_des',
 'Merch_num_state',
 'Merch_num_zip',
 'Merch_des_state',
 'Merch_des_zip',
 'Merch_state_zip',
 'Merch_num_des_state',
 'Merch_num_des_zip',
 'Merch_num_state_zip',
 'Merch_des_state_zip',
 'Merch_all_info',
 'card_Merchnum',
 'card_Merch_description',
 'card_Merch_state',
 'card_Merch_zip',
 'card_Merch_num_des',
 'card_Merch_num_state',
 'card_Merch_num_zip',
 'card_Merch_des_state',
 'card_Merch_des_zip',
 'card_Merch_state_zip',
 'card_Merch_num_des_state',
 'card_Merch_num_des_zip',
 'card_Merch_num_state_zip',
 'card_Merch_des_state_zip',
 'card_Merch_all_info']

In [16]:
def writing_csv(dataframe, filename):
    try:
        dataframe.to_csv('output/' + filename + '.csv',index = False)
    except:
        dataframe.to_csv(filename + '.csv',index = False)

In [17]:
numvar = {}
listvar = []

def num_column_count(dataframe, name):
    
    global numvar, listvar
    
    numvars_new = len(dataframe.columns) - 2 # KK: The excluded 2 are Recnum and fraud_label
    numvar[name] = numvars_new
    
    listvar_new = dataframe.columns.to_list()[2:] # KK: The excluded 2 are Recnum and fraud_label
    listvar += listvar_new

    print("from Dataframe: " + name)
    print("new variables:", numvars_new,"  total variables:", sum(numvar.values()))

In [18]:
df = data.copy()
df['check_date'] = df.Date
df['check_Recnum'] = df.Recnum

del data

In [19]:
%%time
age_and_dow_risk = df[['Recnum', 'Fraud', 'dow_risk']].copy()

# KK: There should be only 2 variables not including "Recnum" and "fraud_label"
# KK: The 2 variables are "age_when_apply" and "dow_risk"
print(age_and_dow_risk.shape)

writing_csv(age_and_dow_risk, '2_0_dow_risk')
num_column_count(age_and_dow_risk, '2_0_dow_risk')
del age_and_dow_risk

(96397, 3)
from Dataframe: 2_0_dow_risk
new variables: 1   total variables: 1
CPU times: total: 109 ms
Wall time: 121 ms


In [20]:
# KK: I will use this instead of "vars"
# KK: I will keep only "Recnum" and "fraud_label". Thus, each of my csv file will not have any variables mixing together.
fraud_label = df.loc[:, ["Recnum", "Fraud"]].copy() 
print(fraud_label.shape)
fraud_label.head()

(96397, 2)


,Recnum,Fraud
47339,47340,0
59516,59517,1
80886,80887,1
89673,89674,0
1103,1104,0


## New Variables

### Benford's Law

In [21]:
bf = df.query("~ Merch_description.str.contains('FEDEX')").copy()
bf['first_dig'] = (bf['Amount']*100).astype(str).str[0]
bf['bin'] = bf['first_dig'].apply(lambda x:'low' if x == '1' else ('low' if x == '2' else 'high'))

In [22]:
def benford_law(column):

    card_bf = bf.groupby([column,'bin']).size().unstack().reset_index()
    card_bf.columns = [column,'n_high','n_low']
    card_bf = card_bf.fillna(1) 

    # calculating R, 1/R, U, n, t, U_smoothed
    c = 3
    n_mid = 15

    card_bf['R'] = (1.096 * card_bf['n_low'] / card_bf['n_high']) # Should be 1
    card_bf['1/R'] = (1/card_bf['R']) # Should also be 1
    card_bf['U'] = card_bf[["R", "1/R"]].max(axis=1) # unusualness

    card_bf['n'] = card_bf['n_high'] + card_bf['n_low']

    card_bf['U_smoothed'] = 1 + (card_bf['U']-1) / (1+np.exp(-(card_bf['n']-n_mid)/c))

    card_Ustar = card_bf.set_index(column)[['U_smoothed']]

    return card_Ustar

card_Ustar = benford_law('Cardnum')
merch_Ustar = benford_law('Merchnum')

In [23]:
final = df.copy() 
final = final.merge(card_Ustar, how = 'left', left_on = 'Cardnum',
                    right_on = card_Ustar.index)
final = final.rename(columns = {'U_smoothed': 'U_smoothed_cardnum'})
final = final.merge(merch_Ustar, how = 'left', left_on = 'Merchnum',
                    right_on = merch_Ustar.index)
final = final.rename(columns = {'U_smoothed': 'U_smoothed_merchnum'})
final['U_smoothed_cardnum'] = final['U_smoothed_cardnum'].replace(np.nan,1)
final['U_smoothed_merchnum'] = final['U_smoothed_merchnum'].replace(np.nan,1)

In [24]:
%%time
ben_law = final[['Recnum', 'Fraud', 'U_smoothed_cardnum', 'U_smoothed_merchnum']].copy()
writing_csv(ben_law, '2_1_benford_law')
num_column_count(ben_law, '2_1_benford_law')

del bf, card_Ustar, merch_Ustar, final, ben_law

from Dataframe: 2_1_benford_law
new variables: 2   total variables: 3
CPU times: total: 234 ms
Wall time: 234 ms


### Velocity/counts and days since

In [25]:
begin=time.time()

vars_2_1_1 = fraud_label.copy() # I will use this instead of "vars"

for i in attributes:
    st = time.time()
    df_1 = df[['Recnum', 'Date', 'Amount', i]]
    df_r = df[['check_Recnum', 'check_date', 'Amount', i]]
    temp = pd.merge(df_1, df_r, left_on = i, right_on = i, suffixes = ('', '_before'))
    
    #day since
    day_since_df = (temp.query("Recnum > check_Recnum")[['Recnum','Date','check_date']]
                    .groupby('Recnum').last()) # last check_date
    mapper_daysince = (day_since_df['Date']-day_since_df.check_date).dt.days
    vars_2_1_1[i + '_day_since'] = vars_2_1_1.Recnum.map(mapper_daysince)
    vars_2_1_1[i + '_day_since'].fillna(365, inplace = True)
    print(f'\n{i}_day_since --->Done')
    
    # Velocity
    for v in [0,1,3,7,14,30]:
        
        # Amount
        count_day_df = (temp[((temp.check_date)>=(temp['Date']-dt.timedelta(v)))
                            &(temp.Recnum>=temp.check_Recnum)]
                        .sort_values(["Recnum", "check_Recnum"], ascending=False)) # Making Recnum Amount be at First row
        mapper_velocity = (count_day_df.groupby('Recnum')["Amount_before"]
                           .agg(["first", "count", "mean", "max", "sum", "std"]))
        mapper_velocity = mapper_velocity.rename(columns = {"mean": "avg", "sum": "total"})
        mapper_velocity["actual/avg"] = mapper_velocity["first"] / mapper_velocity["avg"]
        mapper_velocity["actual/max"] = mapper_velocity["first"] / mapper_velocity["max"]
        mapper_velocity["actual/total"] = mapper_velocity["first"] / mapper_velocity["total"]
        mapper_velocity = mapper_velocity.drop("first", axis=1)
        mapper_velocity = mapper_velocity.rename(columns = lambda func_name: f'{i}_{func_name}_{v}')
        vars_2_1_1 = vars_2_1_1.merge(mapper_velocity, left_on="Recnum", right_index=True)
        
        # Amount Difference
        count_day_df['Amount_diff'] = count_day_df['Amount_before'] - count_day_df['Amount']
        mapper_velocity = (count_day_df.groupby('Recnum')["Amount_diff"]
                           .agg(["mean", "min", "max", "sum", "std"]))
        mapper_velocity = mapper_velocity.rename(columns = {"mean": "avg", "sum": "cum"})
        mapper_velocity = mapper_velocity.rename(columns = lambda func_name: f'{i}_variability_{func_name}_{v}')
        vars_2_1_1 = vars_2_1_1.merge(mapper_velocity, left_on="Recnum", right_index=True)
        
        print(f'{i}_amount_variables_over_past_{str(v)}_days --->Done')

    del day_since_df, count_day_df, df_1, df_r, temp, mapper_daysince, mapper_velocity
    vars_2_1_1 = vars_2_1_1.copy() # solve for fragmented DataFrame

    print(f'Run time for entity {i}--------{time.time()-st:0.2f}s')

writing_csv(vars_2_1_1, '2_2_1_velocity_counts_and_days_since')
num_column_count(vars_2_1_1, '2_2_1_velocity_counts_and_days_since')

print(f'Total run time:{(time.time()-begin)/60:0.2f}min')

# Cannot delete vars_2_1_1 yet as it will be used for 2_1_2 and 2_3
# del vars_2_1_1


Cardnum_day_since --->Done
Cardnum_amount_variables_over_past_0_days --->Done
Cardnum_amount_variables_over_past_1_days --->Done
Cardnum_amount_variables_over_past_3_days --->Done
Cardnum_amount_variables_over_past_7_days --->Done
Cardnum_amount_variables_over_past_14_days --->Done
Cardnum_amount_variables_over_past_30_days --->Done
Run time for entity Cardnum--------5.78s

Merchnum_day_since --->Done
Merchnum_amount_variables_over_past_0_days --->Done
Merchnum_amount_variables_over_past_1_days --->Done
Merchnum_amount_variables_over_past_3_days --->Done
Merchnum_amount_variables_over_past_7_days --->Done
Merchnum_amount_variables_over_past_14_days --->Done
Merchnum_amount_variables_over_past_30_days --->Done
Run time for entity Merchnum--------52.32s

Merch_description_day_since --->Done
Merch_description_amount_variables_over_past_0_days --->Done
Merch_description_amount_variables_over_past_1_days --->Done
Merch_description_amount_variables_over_past_3_days --->Done
Merch_descriptio

card_Merch_num_des_amount_variables_over_past_14_days --->Done
card_Merch_num_des_amount_variables_over_past_30_days --->Done
Run time for entity card_Merch_num_des--------13.69s

card_Merch_num_state_day_since --->Done
card_Merch_num_state_amount_variables_over_past_0_days --->Done
card_Merch_num_state_amount_variables_over_past_1_days --->Done
card_Merch_num_state_amount_variables_over_past_3_days --->Done
card_Merch_num_state_amount_variables_over_past_7_days --->Done
card_Merch_num_state_amount_variables_over_past_14_days --->Done
card_Merch_num_state_amount_variables_over_past_30_days --->Done
Run time for entity card_Merch_num_state--------14.85s

card_Merch_num_zip_day_since --->Done
card_Merch_num_zip_amount_variables_over_past_0_days --->Done
card_Merch_num_zip_amount_variables_over_past_1_days --->Done
card_Merch_num_zip_amount_variables_over_past_3_days --->Done
card_Merch_num_zip_amount_variables_over_past_7_days --->Done
card_Merch_num_zip_amount_variables_over_past_14_day

In [26]:
# relative velocity
start=time.time()

vars_2_1_2 = fraud_label.copy() # I will use this instead of "vars"

for i in attributes:
    for d in ['0','1']:
        for dd in ['3','7','14','30']:
            vars_2_1_2[i+'_count_'+d+'_by_'+dd] = vars_2_1_1[i+'_count_'+d] / (vars_2_1_1[i+'_count_'+dd] / float(dd))
            vars_2_1_2[i+'_total_'+d+'_by_'+dd] = vars_2_1_1[i+'_total_'+d] / (vars_2_1_1[i+'_total_'+dd] / float(dd))
            vars_2_1_2[i+'_vdratio_'+d+'_by_'+dd] = vars_2_1_2[i+'_count_'+d+'_by_'+dd] / (vars_2_1_1[i+'_day_since']+1)

    vars_2_1_2 = vars_2_1_2.copy() # solve for fragmented DataFrame
    print(f'{i}_relative_velocity --->Done')

writing_csv(vars_2_1_2, '2_2_2_relative_velocity')
num_column_count(vars_2_1_2, '2_2_2_relative_velocity')
del vars_2_1_2

print(f'Total run time:{time.time()-start:0.2f}s')

Cardnum_relative_velocity --->Done
Merchnum_relative_velocity --->Done
Merch_description_relative_velocity --->Done
Merch_num_des_relative_velocity --->Done
Merch_num_state_relative_velocity --->Done
Merch_num_zip_relative_velocity --->Done
Merch_des_state_relative_velocity --->Done
Merch_des_zip_relative_velocity --->Done
Merch_state_zip_relative_velocity --->Done
Merch_num_des_state_relative_velocity --->Done
Merch_num_des_zip_relative_velocity --->Done
Merch_num_state_zip_relative_velocity --->Done
Merch_des_state_zip_relative_velocity --->Done
Merch_all_info_relative_velocity --->Done
card_Merchnum_relative_velocity --->Done
card_Merch_description_relative_velocity --->Done
card_Merch_state_relative_velocity --->Done
card_Merch_zip_relative_velocity --->Done
card_Merch_num_des_relative_velocity --->Done
card_Merch_num_state_relative_velocity --->Done
card_Merch_num_zip_relative_velocity --->Done
card_Merch_des_state_relative_velocity --->Done
card_Merch_des_zip_relative_velocity --

In [27]:
# # Maximum Indicator
# begin=time.time()

# # I brought 2.3 to here instead cuz it used data from 2.1
# vars_2_3 = fraud_label.copy() # I will use this instead of "vars"

# # I need both data from 2.1.1 and attributes in df
# vars_2_1_1 = pd.concat([vars_2_1_1, df], axis=1)

# for i in attributes:
    
#     print(i)
#     for v in ['1','3','7','30']:
#         st=time.time()
#         mapper = vars_2_1_1.groupby(i)[i + '_count_' + v].max()
#         vars_2_3['max_count_by_' + i + '_' + v] = vars_2_1_1[i].map(mapper)
#         print(f'max_count_by_{i}_{v} --->Done')

#     print(f'Run time for entity {i}--------{time.time()-st}s')

# writing_csv(vars_2_3, '2_3_maximum_indicator')
# num_column_count(vars_2_3, '2_3_maximum_indicator')
# del vars_2_3

# print(f'Total run time:{(time.time()-begin)/60:0.2f}min')

In [28]:
del vars_2_1_1 # I can drop it after I ran 2.3

### Counts by entities (2.2)

In [29]:
%%time
begin=time.time()

count_attributes = 0 # For naming only

for i in attributes:
    
    count_attributes += 1
    vars_2_2_i = fraud_label.copy() # I will use this instead of "vars"
    # This is actually 2.2
    
    for v in attributes:
        st=time.time()
        if i==v:
            continue
        else:
            df_c=df[['Recnum','Date',i]]
            df_d=df[['check_Recnum','check_date',i,v]]
            temp=pd.merge(df_c,df_d,left_on=i,right_on=i)
        
        #number of unique for particular field
        for t in[0,1,3,7,14,30,60]:
            count_day_df = temp[((temp.check_date)>=(temp.Date-dt.timedelta(t)))
                                & (temp.Recnum>=temp.check_Recnum)]
            col_name = f'{i}_unique_count_for_{v}_{t}'
            mapper = count_day_df.groupby(['Recnum'])[v].nunique()
            vars_2_2_i[col_name] = vars_2_2_i.Recnum.map(mapper)
        
        print(f'Run time for entity {i} in field {v}--------{time.time()-st:0.2f}s')

        del df_c, df_d, temp
        
        vars_2_2_i = vars_2_2_i.copy() # solve for fragmented DataFrame
    
    dataframe_name = '2_3_counts_entities_' + '{0:0>2}'.format(count_attributes) + '_' + i # 2_2_counts_entities_01_ssn
    writing_csv(vars_2_2_i, dataframe_name)
    num_column_count(vars_2_2_i, dataframe_name)
    del vars_2_2_i

    print(f'Total run time:{(time.time()-begin)/60:0.2f}min')

Run time for entity Cardnum in field Merchnum--------6.19s
Run time for entity Cardnum in field Merch_description--------6.94s
Run time for entity Cardnum in field Merch_num_des--------7.39s
Run time for entity Cardnum in field Merch_num_state--------6.55s
Run time for entity Cardnum in field Merch_num_zip--------6.35s
Run time for entity Cardnum in field Merch_des_state--------6.88s
Run time for entity Cardnum in field Merch_des_zip--------7.02s
Run time for entity Cardnum in field Merch_state_zip--------6.18s
Run time for entity Cardnum in field Merch_num_des_state--------6.98s
Run time for entity Cardnum in field Merch_num_des_zip--------7.19s
Run time for entity Cardnum in field Merch_num_state_zip--------6.63s
Run time for entity Cardnum in field Merch_des_state_zip--------6.88s
Run time for entity Cardnum in field Merch_all_info--------6.68s
Run time for entity Cardnum in field card_Merchnum--------6.09s
Run time for entity Cardnum in field card_Merch_description--------6.06s
Run

Run time for entity Merch_num_des in field card_Merch_num_state_zip--------6.73s
Run time for entity Merch_num_des in field card_Merch_des_state_zip--------6.53s
Run time for entity Merch_num_des in field card_Merch_all_info--------6.45s
from Dataframe: 2_3_counts_entities_04_Merch_num_des
new variables: 196   total variables: 3774
Total run time:37.21min
Run time for entity Merch_num_state in field Cardnum--------54.31s
Run time for entity Merch_num_state in field Merchnum--------33.95s
Run time for entity Merch_num_state in field Merch_description--------48.17s
Run time for entity Merch_num_state in field Merch_num_des--------48.40s
Run time for entity Merch_num_state in field Merch_num_zip--------33.49s
Run time for entity Merch_num_state in field Merch_des_state--------48.11s
Run time for entity Merch_num_state in field Merch_des_zip--------48.01s
Run time for entity Merch_num_state in field Merch_state_zip--------32.18s
Run time for entity Merch_num_state in field Merch_num_des_st

Run time for entity Merch_des_zip in field card_Merch_state--------6.86s
Run time for entity Merch_des_zip in field card_Merch_zip--------6.93s
Run time for entity Merch_des_zip in field card_Merch_num_des--------7.23s
Run time for entity Merch_des_zip in field card_Merch_num_state--------7.15s
Run time for entity Merch_des_zip in field card_Merch_num_zip--------7.24s
Run time for entity Merch_des_zip in field card_Merch_des_state--------7.01s
Run time for entity Merch_des_zip in field card_Merch_des_zip--------7.12s
Run time for entity Merch_des_zip in field card_Merch_state_zip--------6.90s
Run time for entity Merch_des_zip in field card_Merch_num_des_state--------7.16s
Run time for entity Merch_des_zip in field card_Merch_num_des_zip--------7.36s
Run time for entity Merch_des_zip in field card_Merch_num_state_zip--------7.29s
Run time for entity Merch_des_zip in field card_Merch_des_state_zip--------7.22s
Run time for entity Merch_des_zip in field card_Merch_all_info--------7.29s
fr

Run time for entity Merch_num_state_zip in field Merch_description--------46.14s
Run time for entity Merch_num_state_zip in field Merch_num_des--------50.62s
Run time for entity Merch_num_state_zip in field Merch_num_state--------32.77s
Run time for entity Merch_num_state_zip in field Merch_num_zip--------32.83s
Run time for entity Merch_num_state_zip in field Merch_des_state--------46.59s
Run time for entity Merch_num_state_zip in field Merch_des_zip--------48.46s
Run time for entity Merch_num_state_zip in field Merch_state_zip--------31.59s
Run time for entity Merch_num_state_zip in field Merch_num_des_state--------48.46s
Run time for entity Merch_num_state_zip in field Merch_num_des_zip--------48.47s
Run time for entity Merch_num_state_zip in field Merch_des_state_zip--------46.98s
Run time for entity Merch_num_state_zip in field Merch_all_info--------48.13s
Run time for entity Merch_num_state_zip in field card_Merchnum--------52.06s
Run time for entity Merch_num_state_zip in field 

Run time for entity card_Merchnum in field card_Merch_num_zip--------1.61s
Run time for entity card_Merchnum in field card_Merch_des_state--------1.92s
Run time for entity card_Merchnum in field card_Merch_des_zip--------2.16s
Run time for entity card_Merchnum in field card_Merch_state_zip--------1.67s
Run time for entity card_Merchnum in field card_Merch_num_des_state--------1.82s
Run time for entity card_Merchnum in field card_Merch_num_des_zip--------1.86s
Run time for entity card_Merchnum in field card_Merch_num_state_zip--------1.69s
Run time for entity card_Merchnum in field card_Merch_des_state_zip--------1.86s
Run time for entity card_Merchnum in field card_Merch_all_info--------1.95s
from Dataframe: 2_3_counts_entities_15_card_Merchnum
new variables: 196   total variables: 5930
Total run time:163.64min
Run time for entity card_Merch_description in field Cardnum--------0.83s
Run time for entity card_Merch_description in field Merchnum--------0.88s
Run time for entity card_Merch

Run time for entity card_Merch_num_des in field Merch_des_state--------0.84s
Run time for entity card_Merch_num_des in field Merch_des_zip--------0.84s
Run time for entity card_Merch_num_des in field Merch_state_zip--------0.85s
Run time for entity card_Merch_num_des in field Merch_num_des_state--------0.95s
Run time for entity card_Merch_num_des in field Merch_num_des_zip--------0.96s
Run time for entity card_Merch_num_des in field Merch_num_state_zip--------0.90s
Run time for entity card_Merch_num_des in field Merch_des_state_zip--------0.82s
Run time for entity card_Merch_num_des in field Merch_all_info--------0.96s
Run time for entity card_Merch_num_des in field card_Merchnum--------0.68s
Run time for entity card_Merch_num_des in field card_Merch_description--------0.83s
Run time for entity card_Merch_num_des in field card_Merch_state--------0.84s
Run time for entity card_Merch_num_des in field card_Merch_zip--------0.83s
Run time for entity card_Merch_num_des in field card_Merch_n

Run time for entity card_Merch_des_state in field card_Merch_num_zip--------0.87s
Run time for entity card_Merch_des_state in field card_Merch_des_zip--------0.82s
Run time for entity card_Merch_des_state in field card_Merch_state_zip--------0.88s
Run time for entity card_Merch_des_state in field card_Merch_num_des_state--------0.95s
Run time for entity card_Merch_des_state in field card_Merch_num_des_zip--------0.86s
Run time for entity card_Merch_des_state in field card_Merch_num_state_zip--------0.87s
Run time for entity card_Merch_des_state in field card_Merch_des_state_zip--------0.78s
Run time for entity card_Merch_des_state in field card_Merch_all_info--------0.86s
from Dataframe: 2_3_counts_entities_22_card_Merch_des_state
new variables: 196   total variables: 7302
Total run time:169.18min
Run time for entity card_Merch_des_zip in field Cardnum--------0.81s
Run time for entity card_Merch_des_zip in field Merchnum--------0.84s
Run time for entity card_Merch_des_zip in field Merc

Run time for entity card_Merch_num_des_zip in field Merch_description--------0.90s
Run time for entity card_Merch_num_des_zip in field Merch_num_des--------0.94s
Run time for entity card_Merch_num_des_zip in field Merch_num_state--------0.88s
Run time for entity card_Merch_num_des_zip in field Merch_num_zip--------1.00s
Run time for entity card_Merch_num_des_zip in field Merch_des_state--------1.04s
Run time for entity card_Merch_num_des_zip in field Merch_des_zip--------0.75s
Run time for entity card_Merch_num_des_zip in field Merch_state_zip--------0.81s
Run time for entity card_Merch_num_des_zip in field Merch_num_des_state--------0.94s
Run time for entity card_Merch_num_des_zip in field Merch_num_des_zip--------0.99s
Run time for entity card_Merch_num_des_zip in field Merch_num_state_zip--------1.05s
Run time for entity card_Merch_num_des_zip in field Merch_des_state_zip--------0.96s
Run time for entity card_Merch_num_des_zip in field Merch_all_info--------1.05s
Run time for entity

Run time for entity card_Merch_all_info in field Merch_des_state_zip--------0.79s
Run time for entity card_Merch_all_info in field Merch_all_info--------0.95s
Run time for entity card_Merch_all_info in field card_Merchnum--------0.82s
Run time for entity card_Merch_all_info in field card_Merch_description--------0.76s
Run time for entity card_Merch_all_info in field card_Merch_state--------0.87s
Run time for entity card_Merch_all_info in field card_Merch_zip--------0.80s
Run time for entity card_Merch_all_info in field card_Merch_num_des--------0.89s
Run time for entity card_Merch_all_info in field card_Merch_num_state--------0.87s
Run time for entity card_Merch_all_info in field card_Merch_num_zip--------0.78s
Run time for entity card_Merch_all_info in field card_Merch_des_state--------0.77s
Run time for entity card_Merch_all_info in field card_Merch_des_zip--------0.85s
Run time for entity card_Merch_all_info in field card_Merch_state_zip--------0.87s
Run time for entity card_Merch_a

In [30]:
print('time to here: ',dt.datetime.now() - start_time)

time to here:  3:08:12.145563


# Summary

In [31]:
# This is the correct one. The one upper said 4204 which is wrong. I made a duplicate file before I fixed the code
# total variables: 4050 -> 4049 as Prof. plus 1 which is either dob_risk or age_at_application. I believe Prof forgot to count it.
print("total variables:", sum(numvar.values()))
numvar

total variables: 8674


{'2_0_dow_risk': 1,
 '2_1_benford_law': 2,
 '2_2_1_velocity_counts_and_days_since': 2291,
 '2_2_2_relative_velocity': 696,
 '2_3_counts_entities_01_Cardnum': 196,
 '2_3_counts_entities_02_Merchnum': 196,
 '2_3_counts_entities_03_Merch_description': 196,
 '2_3_counts_entities_04_Merch_num_des': 196,
 '2_3_counts_entities_05_Merch_num_state': 196,
 '2_3_counts_entities_06_Merch_num_zip': 196,
 '2_3_counts_entities_07_Merch_des_state': 196,
 '2_3_counts_entities_08_Merch_des_zip': 196,
 '2_3_counts_entities_09_Merch_state_zip': 196,
 '2_3_counts_entities_10_Merch_num_des_state': 196,
 '2_3_counts_entities_11_Merch_num_des_zip': 196,
 '2_3_counts_entities_12_Merch_num_state_zip': 196,
 '2_3_counts_entities_13_Merch_des_state_zip': 196,
 '2_3_counts_entities_14_Merch_all_info': 196,
 '2_3_counts_entities_15_card_Merchnum': 196,
 '2_3_counts_entities_16_card_Merch_description': 196,
 '2_3_counts_entities_17_card_Merch_state': 196,
 '2_3_counts_entities_18_card_Merch_zip': 196,
 '2_3_counts_e

In [32]:
listvar

['dow_risk',
 'U_smoothed_cardnum',
 'U_smoothed_merchnum',
 'Cardnum_day_since',
 'Cardnum_count_0',
 'Cardnum_avg_0',
 'Cardnum_max_0',
 'Cardnum_total_0',
 'Cardnum_std_0',
 'Cardnum_actual/avg_0',
 'Cardnum_actual/max_0',
 'Cardnum_actual/total_0',
 'Cardnum_variability_avg_0',
 'Cardnum_variability_min_0',
 'Cardnum_variability_max_0',
 'Cardnum_variability_cum_0',
 'Cardnum_variability_std_0',
 'Cardnum_count_1',
 'Cardnum_avg_1',
 'Cardnum_max_1',
 'Cardnum_total_1',
 'Cardnum_std_1',
 'Cardnum_actual/avg_1',
 'Cardnum_actual/max_1',
 'Cardnum_actual/total_1',
 'Cardnum_variability_avg_1',
 'Cardnum_variability_min_1',
 'Cardnum_variability_max_1',
 'Cardnum_variability_cum_1',
 'Cardnum_variability_std_1',
 'Cardnum_count_3',
 'Cardnum_avg_3',
 'Cardnum_max_3',
 'Cardnum_total_3',
 'Cardnum_std_3',
 'Cardnum_actual/avg_3',
 'Cardnum_actual/max_3',
 'Cardnum_actual/total_3',
 'Cardnum_variability_avg_3',
 'Cardnum_variability_min_3',
 'Cardnum_variability_max_3',
 'Cardnum_varia

In [33]:
df_numvar = pd.DataFrame(numvar, index=["count_variables"]).T.reset_index()
df_numvar.loc['Total'] = df_numvar.sum(numeric_only=True)
df_numvar.columns = ['filename', 'count_variables']

df_numvar.to_csv('output/3_count_variables.csv')
df_numvar

,filename,count_variables
0,2_0_dow_risk,1.0
1,2_1_benford_law,2.0
2,2_2_1_velocity_counts_and_days_since,2291.0
3,2_2_2_relative_velocity,696.0
4,2_3_counts_entities_01_Cardnum,196.0
5,2_3_counts_entities_02_Merchnum,196.0
6,2_3_counts_entities_03_Merch_description,196.0
7,2_3_counts_entities_04_Merch_num_des,196.0
8,2_3_counts_entities_05_Merch_num_state,196.0
9,2_3_counts_entities_06_Merch_num_zip,196.0


In [34]:
df_listvar = pd.DataFrame(listvar, columns = ["variables"]).drop_duplicates()[2:].reset_index(drop = True)

df_listvar.to_csv('output/3_list_variables.csv')
df_listvar

,variables
0,U_smoothed_merchnum
1,Cardnum_day_since
2,Cardnum_count_0
3,Cardnum_avg_0
4,Cardnum_max_0
...,...
8667,card_Merch_all_info_unique_count_for_card_Merc...
8668,card_Merch_all_info_unique_count_for_card_Merc...
8669,card_Merch_all_info_unique_count_for_card_Merc...
8670,card_Merch_all_info_unique_count_for_card_Merc...


In [35]:
print('duration: ',dt.datetime.now() - start_time)
# takes about an hour on my new laptop without deduping.
# takes about 2 hours on my old laptop if I don't remove duplicate columns. Then I can run the separate remove duplicates code.

duration:  3:08:12.443565
